# Get subway station data and conpute distances to Citibike Stations

In [ ]:
import pandas as pd

In [2]:
#location of subway station data
url = "http://web.mta.info/developers/data/nyct/subway/Stations.csv"

In [3]:
#store subway station data
station_data = pd.read_csv(url)
station_data.head(5)

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN


In [4]:
#access citibike data
filepath = "resources/full-data.csv"
bike_data = pd.read_csv(filepath)
bike_data.head(5)

,Unnamed: 0,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1
1,1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
2,2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1
3,3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1
4,4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1


In [5]:
#create list of citibike stations
bike_stations = bike_data.drop_duplicates(subset="start station id")
bike_stations.head(5)

,Unnamed: 0,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1
1,1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
2,2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1
3,3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1
4,4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1


In [6]:
#create dataframe of just citibike station id and coordinates
bike_stations_cleaned = bike_stations[["start station id","start station latitude","start station longitude"]]
bike_stations_cleaned.head(5)

,start station id,start station latitude,start station longitude
0,3160.0,40.778968,-73.973747
1,519.0,40.751873,-73.977706
2,3171.0,40.785247,-73.976673
3,504.0,40.732219,-73.981656
4,229.0,40.727434,-73.993790


In [7]:
#import geopy distance function to determine distances based on lat/lon
from geopy import distance

In [8]:
#for each citibike station, loop through all subway stations and determine the distance between the two.
#append that distance to a list, then find the min value to determine the distance to the clostest subway
#append that value to a list
distance_to_subway = []
for x in range(len(bike_stations_cleaned)):
    bike_lat = bike_stations_cleaned.iloc[x,1]
    bike_lon = bike_stations_cleaned.iloc[x,2]
    bike_coord = (bike_lat, bike_lon)
    subway_distances = []
    for y in range(len(station_data)):
        subway_lat = station_data.iloc[y,9]
        subway_lon = station_data.iloc[y,10]
        subway_coord = (subway_lat, subway_lon)
        station_distance = (distance.distance(bike_coord, subway_coord).feet)
        subway_distances.append(station_distance)
    distance_to_subway.append(min(subway_distances))
    print(f"{x+1} of {len(bike_stations_cleaned)}")    

1 of 1236
2 of 1236
3 of 1236
4 of 1236
5 of 1236
6 of 1236
7 of 1236
8 of 1236
9 of 1236
10 of 1236
11 of 1236
12 of 1236
13 of 1236
14 of 1236
15 of 1236
16 of 1236
17 of 1236
18 of 1236
19 of 1236
20 of 1236
21 of 1236
22 of 1236
23 of 1236
24 of 1236
25 of 1236
26 of 1236
27 of 1236
28 of 1236
29 of 1236
30 of 1236
31 of 1236
32 of 1236
33 of 1236
34 of 1236
35 of 1236
36 of 1236
37 of 1236
38 of 1236
39 of 1236
40 of 1236
41 of 1236
42 of 1236
43 of 1236
44 of 1236
45 of 1236
46 of 1236
47 of 1236
48 of 1236
49 of 1236
50 of 1236
51 of 1236
52 of 1236
53 of 1236
54 of 1236
55 of 1236
56 of 1236
57 of 1236
58 of 1236
59 of 1236
60 of 1236
61 of 1236
62 of 1236
63 of 1236
64 of 1236
65 of 1236
66 of 1236
67 of 1236
68 of 1236
69 of 1236
70 of 1236
71 of 1236
72 of 1236
73 of 1236
74 of 1236
75 of 1236
76 of 1236
77 of 1236
78 of 1236
79 of 1236
80 of 1236
81 of 1236
82 of 1236
83 of 1236
84 of 1236
85 of 1236
86 of 1236
87 of 1236
88 of 1236
89 of 1236
90 of 1236
91 of 1236
92 of 12

695 of 1236
696 of 1236
697 of 1236
698 of 1236
699 of 1236
700 of 1236
701 of 1236
702 of 1236
703 of 1236
704 of 1236
705 of 1236
706 of 1236
707 of 1236
708 of 1236
709 of 1236
710 of 1236
711 of 1236
712 of 1236
713 of 1236
714 of 1236
715 of 1236
716 of 1236
717 of 1236
718 of 1236
719 of 1236
720 of 1236
721 of 1236
722 of 1236
723 of 1236
724 of 1236
725 of 1236
726 of 1236
727 of 1236
728 of 1236
729 of 1236
730 of 1236
731 of 1236
732 of 1236
733 of 1236
734 of 1236
735 of 1236
736 of 1236
737 of 1236
738 of 1236
739 of 1236
740 of 1236
741 of 1236
742 of 1236
743 of 1236
744 of 1236
745 of 1236
746 of 1236
747 of 1236
748 of 1236
749 of 1236
750 of 1236
751 of 1236
752 of 1236
753 of 1236
754 of 1236
755 of 1236
756 of 1236
757 of 1236
758 of 1236
759 of 1236
760 of 1236
761 of 1236
762 of 1236
763 of 1236
764 of 1236
765 of 1236
766 of 1236
767 of 1236
768 of 1236
769 of 1236
770 of 1236
771 of 1236
772 of 1236
773 of 1236
774 of 1236
775 of 1236
776 of 1236
777 of 1236
778 

In [9]:
#add distances to the nearest subway to the citibike station dataframe
bike_stations_cleaned["distance to subway"] = distance_to_subway

<ipython-input-9-200d8f516dc4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bike_stations_cleaned["distance to subway"] = distance_to_subway


In [30]:
bike_stations_cleaned.head(5)

,start station id,start station latitude,start station longitude,distance to subway
0,3160.0,40.778968,-73.973747,1002.045859
1,519.0,40.751873,-73.977706,240.335886
2,3171.0,40.785247,-73.976673,1017.691651
3,504.0,40.732219,-73.981656,461.137755
4,229.0,40.727434,-73.993790,603.613380


In [33]:
bike_stations_cleaned.to_csv("resources/citibike_stations.csv")

In [25]:
#for each subway station, loop through all citibike stations and determine the distance between the two.
#append that distance to a list, then find the min value to determine the distance to the clostest citibike station
#append that value to a list
distance_to_bike = []
for x in range(len(station_data)):
    sub_lat = station_data.iloc[x,9]
    sub_lon = station_data.iloc[x,10]
    sub_coord = (sub_lat, sub_lon)
    bike_distances = []
    for y in range(len(bike_stations_cleaned)):
        bike_lat = bike_stations_cleaned.iloc[y,1]
        bike_lon = bike_stations_cleaned.iloc[y,2]
        bike_coord = (bike_lat, bike_lon)
        bike_distance = (distance.distance(bike_coord, sub_coord).feet)
        bike_distances.append(bike_distance)
    distance_to_bike.append(min(bike_distances))
    print(f"{x+1} of {len(station_data)} ({sub_coord})")

1 of 496 ((40.775036, -73.91203399999999))
2 of 496 ((40.770258, -73.917843))
3 of 496 ((40.766779, -73.921479))
4 of 496 ((40.76182, -73.925508))
5 of 496 ((40.756803999999995, -73.929575))
6 of 496 ((40.752882, -73.932755))
7 of 496 ((40.76266, -73.967258))
8 of 496 ((40.764811, -73.973347))
9 of 496 ((40.764664, -73.98065799999999))
10 of 496 ((40.759901, -73.984139))
11 of 496 ((40.754672, -73.986754))
12 of 496 ((40.749567, -73.98795))
13 of 496 ((40.745494, -73.988691))
14 of 496 ((40.741303, -73.98934399999999))
15 of 496 ((40.735735999999996, -73.99056800000001))
16 of 496 ((40.730328, -73.99262900000001))
17 of 496 ((40.724329, -73.997702))
18 of 496 ((40.719527, -74.001775))
19 of 496 ((40.718383, -74.00046))
20 of 496 ((40.713282, -74.006978))
21 of 496 ((40.710668, -74.01102900000001))
22 of 496 ((40.70722, -74.01334200000001))
23 of 496 ((40.703087, -74.01299399999999))
24 of 496 ((40.6941, -73.991777))
25 of 496 ((40.69218, -73.98594200000001))
26 of 496 ((40.690635, -73.

209 of 496 ((40.600066, -73.761353))
210 of 496 ((40.603995, -73.75540500000001))
211 of 496 ((40.874811, -73.878855))
212 of 496 ((40.873244, -73.88713800000001))
213 of 496 ((40.866978, -73.893509))
214 of 496 ((40.861296, -73.897749))
215 of 496 ((40.856093, -73.900741))
216 of 496 ((40.85041, -73.905227))
217 of 496 ((40.8459, -73.910136))
218 of 496 ((40.839306, -73.9134))
219 of 496 ((40.833771, -73.91844))
220 of 496 ((40.827905, -73.925651))
221 of 496 ((40.830135, -73.938209))
222 of 496 ((40.754203000000004, -73.942836))
223 of 496 ((40.759145000000004, -73.95326))
224 of 496 ((40.764629, -73.966113))
225 of 496 ((40.763971999999995, -73.97745))
226 of 496 ((40.758663, -73.981329))
227 of 496 ((40.754222, -73.984569))
228 of 496 ((40.749719, -73.98782299999999))
229 of 496 ((40.742878000000005, -73.99282099999999))
230 of 496 ((40.738228, -73.996209))
231 of 496 ((40.725297, -73.996204))
232 of 496 ((40.718267, -73.993753))
233 of 496 ((40.723402, -73.989938))
234 of 496 ((40

415 of 496 ((40.704817, -74.014065))
416 of 496 ((40.692403999999996, -73.990151))
417 of 496 ((40.903125, -73.85061999999999))
418 of 496 ((40.898379, -73.854376))
419 of 496 ((40.893193, -73.857473))
420 of 496 ((40.888022, -73.860341))
421 of 496 ((40.883895, -73.862633))
422 of 496 ((40.87785, -73.866256))
423 of 496 ((40.871356, -73.867164))
424 of 496 ((40.865462, -73.867352))
425 of 496 ((40.857192, -73.867615))
426 of 496 ((40.848828000000005, -73.86845699999999))
427 of 496 ((40.841894, -73.873488))
428 of 496 ((40.840295000000005, -73.880049))
429 of 496 ((40.837288, -73.887734))
430 of 496 ((40.829993, -73.891865))
431 of 496 ((40.824073, -73.893064))
432 of 496 ((40.822181, -73.896736))
433 of 496 ((40.819585, -73.90177))
434 of 496 ((40.81649, -73.90780699999999))
435 of 496 ((40.816109000000004, -73.917757))
436 of 496 ((40.81841, -73.92671800000001))
437 of 496 ((40.823879999999996, -73.93647))
438 of 496 ((40.820421, -73.936245))
439 of 496 ((40.814229, -73.94077))
440 

In [27]:
station_data_new = station_data.copy()

In [28]:
#add distances to the nearest citibike to the subway station dataframe
station_data_new["Distance to Bike Station"] = distance_to_bike

In [32]:
station_data_new.head(5)

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Distance to Bike Station
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN,170.318168
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN,199.183685
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN,318.580062
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN,143.119239
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN,106.119821


In [35]:
station_data_new.to_csv("resources/subway_station_data.csv")